In [ ]:
""""
OANDAデータ→修正BBデータ変換

""""

%matplotlib inline

import glob
import csv

ORG_PRICE_DATA_DIR = "../money/data" 

PRICE_DATA_FILE = "./data.tsv"

HEADER = ["highBid", "closeAsk", "lowBid", "lowAsk", "openAsk", "complete", "highAsk", "time", "closeBid", "openBid", "volume"]

files = glob.glob(ORG_PRICE_DATA_DIR + "/USD_JPY/*/*/*")
# files = glob.glob(ORG_PRICE_DATA_DIR + "/USD_JPY/2015/01/0*")
price_idx = HEADER.index("closeAsk")
time_idx = HEADER.index("time")
with open(PRICE_DATA_FILE, "w") as ofp:
    for f in files:
        date = f.split("/")
        year = date[-3]
        month = date[-2]
        day = date[-1].split(".")[0]
        writer = csv.writer(ofp, delimiter = "\t", lineterminator = "\n")
        with open(f, "r") as ifp:
            reader = csv.reader(ifp, delimiter = "\t")
            for line in reader:
                writer.writerow(["-".join([year, month, day]) + " " + line[time_idx], line[price_idx]])

print("Success")


In [2]:
#""""
#修正BB係数算出
#
#入力データ
#　フォーマット：TSV
#　形式　　　　：日時(yyyy-mm-dd hh:MM:ss), 価格
#
#""""

import numpy as np
import csv
from datetime import datetime as dt

DATA = "./data.tsv"
OUTPUT_FILE = "./threshold.tsv"

MOV_AVG_SPAN = 13
STD_SPAN = 20

DATE_IDX = 0
PRICE_IDX = 1

COEF_WIDTH = 5.0
COEF_STEP = 1.0

def is_calc_coef(prev_date, date):
    if prev_date != None and prev_date.day != date.day:     # 日
    # if prev_date != None and prev_date.month != date.month:     # 月
        return True
    else:
        return False

def calculate_coef(data, coef_range):
    max_profit = 0
    result_max_coef = None
    result_min_coef = None
    for max_coef in coef_range:
        for min_coef in coef_range:
            position = 0.0
            asset = 0.0
            for d in data:
                date = d["date"]
                price = d["price"]
                mavg = d["mavg"]
                std = d["std"]
                is_update = False
                
                min_th = mavg - std * min_coef
                max_th = mavg + std * max_coef
                # print("Min TH\t" + str(min_th) + "\t" + "MAX TH\t" + str(max_th))
                # print("Mean Avg\t" + str(mavg) + "\t" + "STD\t" + str(std) + "\t"
                #        "Min Coef\t" + str(min_coef) + "\t" + "Max Coef\t" + str(max_coef))
                if price < min_th:
                    position += 1
                    asset = asset - price
                    is_update = True
                if price > max_th:
                    position -= 1
                    asset = asset + price
                    is_update = True

                    
                # print("Mean Avg\t" + str(mavg) + "\t" + "STD\t" + str(std) + "\t"
                #          "Min Coef\t" + str(min_coef) + "\t" + "Max Coef\t" + str(max_coef))
                # # # print(mavg - std * min_coef)
                # print("Min TH\t" + str(min_th) + "\t" + "MAX TH\t" + str(max_th))
                # profit = asset + position * price
                # print("Profit\t" + str(profit) + "\t" + "Asset\t" + str(asset) + "\t" + "Position\t" + str(position))
                        
                    
                if is_update:
                    profit = asset + position * price
                    if profit > max_profit:
                        result_max_coef = max_coef
                        result_min_coef = min_coef
                        max_profit = profit

    print("Date\t" + str(date) + "\t"
             + "Max Profit\t" + str(max_profit) + "\t"
             + "Max Coef\t" + str(result_max_coef) + "\t"
             + "Min Coef\t" + str(result_min_coef))
    return result_max_coef, result_min_coef
            
with open(OUTPUT_FILE, "w") as ofp, open(DATA, "r") as ifp:
    writer = csv.writer(ofp, delimiter = "\t", lineterminator = "\n")
    reader = csv.reader(ifp, delimiter = "\t")
    mavg_data = np.array([])
    std_data = np.array([])
    eval_data = []
    coef_range = np.arange(0, COEF_WIDTH, COEF_STEP)
    prev_date = None
    for v in reader:
        skip_flag = False
        date = dt.strptime(v[DATE_IDX], '%Y-%m-%d %H:%M:%S')
        price = float(v[PRICE_IDX])
        if len(mavg_data) >= MOV_AVG_SPAN:
            mavg_data = np.hstack((mavg_data[1:], np.array(price))) 
        else:
            mavg_data = np.hstack((mavg_data, price))
            skip_flag = True
        if len(std_data) >= STD_SPAN:
            std_data = np.hstack((std_data[1:], np.array(price))) 
        else:
            std_data = np.hstack((std_data, price))        
            skip_flag = True
        if skip_flag:
            continue
        mavg = np.average(mavg_data)
        std = np.std(std_data)
        if is_calc_coef(prev_date, date):
            max_coef, min_coef = calculate_coef(eval_data, coef_range)
            writer.writerow([date, min_coef, max_coef])
            eval_data = []
        eval_data.append({"date": date, "price": price, "mavg": mavg, "std": std})
        prev_date = date
            

print("Success")
        


Date	2015-01-02 21:59:55	Max Profit	1314.5310000000172	Max Coef	0.0	Min Coef	4.0
Date	2015-01-05 23:59:30	Max Profit	3554.709999998915	Max Coef	0.0	Min Coef	4.0
Date	2015-01-06 23:59:35	Max Profit	3953.047999999486	Max Coef	0.0	Min Coef	4.0
Date	2015-01-07 23:59:55	Max Profit	1905.189000000537	Max Coef	4.0	Min Coef	0.0
Date	2015-01-08 23:59:45	Max Profit	892.3959999999497	Max Coef	4.0	Min Coef	0.0
Date	2015-01-09 21:59:55	Max Profit	3219.305000001681	Max Coef	0.0	Min Coef	4.0
Date	2015-01-11 23:59:50	Max Profit	47.617999999976746	Max Coef	0.0	Min Coef	4.0
Date	2015-01-12 23:59:55	Max Profit	1915.1809999998077	Max Coef	0.0	Min Coef	4.0
Date	2015-01-13 23:59:55	Max Profit	3644.5269999982556	Max Coef	0.0	Min Coef	4.0
Date	2015-01-14 23:59:50	Max Profit	3746.9640000000945	Max Coef	0.0	Min Coef	4.0
Date	2015-01-15 23:59:50	Max Profit	6532.40200000233	Max Coef	0.0	Min Coef	4.0
Date	2015-01-16 21:59:55	Max Profit	5132.971999997739	Max Coef	4.0	Min Coef	0.0
Date	2015-01-18 23:59:55	Max Profit	

Date	2015-04-30 23:59:55	Max Profit	3250.1070000008913	Max Coef	4.0	Min Coef	0.0
Date	2015-05-01 20:59:45	Max Profit	1081.3509999999078	Max Coef	4.0	Min Coef	0.0
Date	2015-05-03 23:59:45	Max Profit	11.704999999998108	Max Coef	0.0	Min Coef	2.0
Date	2015-05-04 23:59:50	Max Profit	276.5709999995015	Max Coef	3.0	Min Coef	0.0
Date	2015-05-05 23:59:30	Max Profit	1187.29900000029	Max Coef	0.0	Min Coef	4.0
Date	2015-05-06 23:59:55	Max Profit	1331.9539999999688	Max Coef	0.0	Min Coef	4.0
Date	2015-05-07 23:59:55	Max Profit	1670.6880000000237	Max Coef	4.0	Min Coef	0.0
Date	2015-05-08 20:59:40	Max Profit	885.1800000005751	Max Coef	0.0	Min Coef	4.0
Date	2015-05-10 23:59:55	Max Profit	20.4299999999821	Max Coef	2.0	Min Coef	0.0
Date	2015-05-11 23:59:40	Max Profit	719.4749999992782	Max Coef	4.0	Min Coef	0.0
Date	2015-05-12 23:59:55	Max Profit	813.8249999993714	Max Coef	0.0	Min Coef	4.0
Date	2015-05-13 23:59:50	Max Profit	1631.8140000002459	Max Coef	0.0	Min Coef	4.0
Date	2015-05-14 23:59:55	Max Profit	

In [6]:
#""""
#修正BB係数算出
#
#入力データ
#　フォーマット：TSV
#　形式　　　　：日時(yyyy-mm-dd hh:MM:ss), 価格
# 
#閾値データ
#　フォーマット：TSV
#　形式　　　　：日時(yyyy-mm-dd hh:MM:ss), 下限の閾値, 上限の閾値
#
#""""

import numpy as np
import csv
import datetime

DATA = "./data.tsv"
TH_FILE = "./threshold.tsv"

with open(TH_FILE, "r") as th_fp, open(DATA, "r") as data_fp: # データファイルと閾値ファイルを開く
    reader = csv.reader(th_fp, delimiter = "\t") # 閾値ファイルを読み込み
    coef_dict = {}
    for d in reader:
        date = datetime.datetime.strptime(d[0], '%Y-%m-%d %H:%M:%S') # レコードから日付を取得
        min_coef = float(d[1]) # レコードからmin_coefを取得
        max_coef = float(d[2]) # レコードからmax_coefを取得
        coef_dict[datetime.datetime(date.year, date.month, date.day)] = {"min": min_coef, "max": max_coef} # 辞書型として、coef_dictを作成
    
    # print(coef_dict)
    
    reader = csv.reader(data_fp, delimiter = "\t") # データファイルを読み込み
    asset = 0.0
    position = 0.0
    for d in reader: # CSVデータの分だけ繰り返し
        date = datetime.datetime.strptime(d[0], '%Y-%m-%d %H:%M:%S') # 日付抽出
        price = float(d[1]) # 価格抽出
        coef_date = datetime.datetime(date.year, date.month, date.day) - datetime.timedelta(days = 1) # 上のdateから１日ひいている？
        if not coef_date in coef_dict:
            continue
        if price > coef_dict[coef_date]["max"]: # coefmaxより価格が大きかったら購入？
            if asset > price: # 
                new_position = float(asset // price)
                position += new_position
                asset -= new_position * price
            else:
                position += 1
                asset -= price
        elif price < coef_dict[coef_date]["min"]:
            if position > 0:
                asset += position * price
                position = 0
            else:
                position -= 1
                asset += price
    
    # print("Asset\t" + str(asset) + "\t" + "Position\t" + str(position))

    asset += position * price
    print("Asset\t" + str(asset))
    

Asset	4392285.8409986645
